# Landsat Collection 2 downloader
michael.brechbuehler@eawag.ch

This notebook uses the [landsatxplore](https://github.com/yannforget/landsatxplore) python package to download Landsat Collection 2 scenes directly from USGS.

### Setup environment
To setup a new environment (using conda):
> conda create --name landsatxplore -c conda-forge python=3 notebook nb_conda_kernels geopandas hvplot cartopy geoviews

To setup a new environment (using conda and environment.yml):
> conda env create -n landsatxplore --file environment.yml

### Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import shapely
import hvplot.pandas
from tqdm import tqdm
from pathlib import Path
import tarfile
from contextlib import closing

# import landsatxplore
import landsatxplore
from landsatxplore.earthexplorer import EarthExplorer
from landsatxplore.api import API

## 1. Search query

### USGS EROS credentials
To allow queries and file downloads from the USGS EROS webserver a free account is necessary (https://ers.cr.usgs.gov/register)

In [ ]:
# Your USGS  credentials
username = "..."
password = "..."

Now we can set the desired search parameters for our Landsat query. We also have to define which Landsat collection to use.

| Dataset Name | Dataset ID |
| :-- | -- |
| Landsat 5 TM Collection 2 Level 1 | `landsat_tm_c2_l1` |
| Landsat 5 TM Collection 2 Level 2 | `landsat_tm_c2_l2` |
| Landsat 7 ETM+ Collection 2 Level 1 | `landsat_etm_c2_l1` |
| Landsat 7 ETM+ Collection 2 Level 2 | `landsat_etm_c2_l2` |
| Landsat 8 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 8 Collection 2 Level 2 | `landsat_ot_c2_l2` |
| Landsat 9 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 9 Collection 2 Level 2 | `landsat_ot_c2_l2` |

### Search parameters

In [ ]:
# Search parameters
bbox = (6.117096, 46.189198, 6.938324 ,46.544463) # (lonmin, latmin, lonmax, latmax)

search_params = {
    'dataset': 'landsat_ot_c2_l2',
    'bbox': bbox, 
    'start_date': '2023-01-01',
    'end_date': '2023-12-31',
    'months': [1,2,3,4,5,6,7,8,9,10,11,12],
    'max_cloud_cover': 100,
    'max_results': 1000, # Defaults to 100 if not specified
}

### Plot ROI

In [ ]:
# Plot bounding box on map
bbox_poly = shapely.geometry.box(*bbox, ccw=True)
df_bbox = gpd.GeoDataFrame([], geometry=[bbox_poly])

plot_bbox = df_bbox.hvplot(title='ROI bounding box', geo=True, 
                           fill_alpha=0.2, color='red', line_color='red', 
                           #tiles='CartoLight'
                           tiles='OSM', label='ROI'
                          )

plot_bbox

### Start search query

In [ ]:
# initialize a new API instance
api = API(username, password)

# search for Landsat 8-9 L2C2 scenes
scenes = api.search(**search_params)

# logout
api.logout()

# create a GeoDataFrame from the returned metadata
# to see all metadata check "scenes" dict
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id', 'data_type', 'wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date', 'spatial_coverage']]
df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
df_scenes['satellite'] = df_scenes.satellite.apply(lambda x: f'Landsat-{x}')
df_scenes['tile'] = df_scenes.display_id.apply(lambda x: x.split('_')[2][:3]+'/'+x.split('_')[2][3:])
gdf_scenes = gpd.GeoDataFrame(df_scenes.drop(columns='spatial_coverage'), geometry=df_scenes.spatial_coverage, crs='EPSG:4326')

gdf_scenes

### Plot available tiles over ROI

In [ ]:
gdf_tiles = gdf_scenes.groupby('tile').first().reset_index()[['tile', 'wrs_path', 'wrs_row', 'geometry']].set_crs('EPSG:4326')
gdf_labels = gdf_tiles.set_geometry(gdf_tiles.centroid)

plot_labels = gdf_labels.to_crs("EPSG:3857").assign(x=lambda df: df.geometry.x, y=lambda df: df.geometry.y).hvplot.labels(text="tile", x="x", y="y", text_color='white')

plot_tiles = gdf_tiles.hvplot(title='Available tiles over ROI', geo=True,
                              fill_alpha=0.3, line_color='blue',
                              height=800, tiles='OSM', label='Landsat tile'
                             ) * plot_bbox * plot_labels

plot_tiles

### Plot all available scenes

In [ ]:
plot_scenes = df_scenes.hvplot.line(title=f'Available Landsat C2 scenes over ROI\n{search_params["start_date"]} to {search_params["end_date"]}, n={df_scenes.shape[0]}',
    x='acquisition_date', y='cloud_cover', color='grey', grid=True) *\
                df_scenes.hvplot.scatter(x='acquisition_date', y='cloud_cover', c='satellite',
                                       hover_cols=['tile', 'acquisition_date'],
                                       xlabel='Acq. date', ylabel='Cloud cover (%)', marker='o')
plot_scenes

## 2. Download scenes
### Filter and select scene IDs

In [ ]:
# Select scene ids to download from dataframe
#ids = df_scenes.display_id.values # all
df_scenes_filt = df_scenes.loc[(df_scenes.cloud_cover < 2) & (df_scenes.tile=='196/028')] # filter by cloudcover and tile
ids = df_scenes_filt.display_id.values
df_scenes_filt

### Download .tar files

In [ ]:
# set output directory
path_output_dir = Path(rf"E:/{search_params['dataset']}")

# initialize the API
ee = EarthExplorer(username, password)

# download the scenes
for id in tqdm(ids, total=len(ids), desc="Total download progress", position=-1):
    path_output_file = path_output_dir.joinpath(id+'.tar')
    try:
        ee.download(id, output_dir=path_output_dir)
        print('{} successful'.format(id))

  # aditional error handling
    except:
        if path_output_file.exists():
            print(f'{id} error but file exists')
        else:
            print(f'{id} error')

ee.logout()

### Extract .tar files
Unpack compressed files and delete .tar files.

In [ ]:
paths_output = list(path_output_dir.glob('*.tar'))

for path in tqdm(paths_output, desc="Total extraction progress"):
    with closing(tarfile.open(path)) as fl:
        path_output_folder = path_output_dir.joinpath(path.name.split('.')[0])
        path_output_folder.mkdir(parents=True, exist_ok=True)
        fl.extractall(path_output_folder)
    path.unlink()

## Remove unwanted bands
Delete all files associated with surface reflectance bands based on "SR_" string.

In [ ]:
paths_output = list(path_output_dir.rglob('*SR_*'))

for path in paths_output:
    path.unlink()